# Notes & Code

## Chapter: Scaling with generators

Let's start with iterations, iteration is simply pass over each element in a sequence doing some operation on that. It's largelly used in for loops. But when a for loop is used, what happens under the hood? <br>
So actually python uses a built-in function called `iter()` transforming a sequence into a `iterator`, what points to next element each iteration. <br> 

- iterator: An iterator is like a moving pointer over the collections.

In [2]:
numbers = [1, 2, 3, 4, 5]

for num in numbers: print(num)

1
2
3
4
5


In [5]:
numbers = [1, 2, 3, 4, 5]

numbers_iter = iter(numbers)

for num in numbers_iter: print(num)
    
print(f'numbers type: {type(numbers)}')
print(f'numbers_iter type {type(numbers_iter)}')

1
2
3
4
5
numbers type: <class 'list'>
numbers_iter type <class 'list_iterator'>


So I have this situation for example in a for loop what python actually does is get some sequence and transform it into a iterator. And how python do that?<br>
Simple!<br>

There is a built-in function called iter() that get a sequence and return a iterator like showed previously. And how does iter() get the iterator?<br>

Many ways to do that but one specific is define the magic dunder method `__iter__`. In this point we have to define the difference between iterator and iterable.<br>

- iterable: Are objects that return an iterator when passed through iter() function.
- iterator: Are the returns of iter() function, an object that have give one element by time.

In [1]:
names = ['Tom', 'Shelly',' Garth']
names_it = iter(names)

In [5]:
next(names_it)

StopIteration: 

Analisando mais detalhadamente a operação iter() sobre um objeto iterável, podemos utilizar outra built-in function chamada next() para acessar o próximo elemento do iterador. aplicando next() sucessivas vezes, consigo acessar todos os elementos da sequencia, até que ao não ter mais elementos, ocorre um raise exception de um erro do tipo `StopIteration`.

In [57]:
names = ['Tom', 'Shelly',' Garth']
names_it = iter(names)

In [21]:
# neste caso, tenho também a presença de um valor default onde ao invés de um raise exception, terei obterei o valor default.
# este valor default continua ocorrendo indefinidamente ao acessar o iterator desta forma.

next(names_it, "Rodrigo")

'Rodrigo'

Vamos agora, começar a analisar onde esta definição de iteradores e iteráveis podem trazer ganho em sua utilização.

In [25]:
# Definição de uma função que calcula uma lista de quadrados de numeros, um a um recebendo como parametro o limite superior 
# da lista

def calculate_squares(__max):
    
    squares = list()
    
    for num in range(__max):
        
        squares.append(num ** 2)
        
    return squares

In [28]:
calculate_squares(4)

[0, 1, 4, 9]

Reparamos que neste caso passando para função o valor máximo de elementos que quero calcular o quadrado, é retornado uma lista de valores. Isto significa que preciso ocupar espaço na memória com uma lista de quadrados calculados. Ou seja, dependendo do numero de elementos este valor pode rapidamente explodir a memória. Imagine se quiser uma lista com 1 milhão de quadrados ou mais.<br>
Como comentado anteriormente, se uma classe definir o método mágico `__iter__` e o método `__next__` é possível obter um iterador passando este objeto pela função iter() e neste caso, também como visto anteriormente, não seria criada uma lista de valores e sim ocorreria a obtenção de um valor da sequencia por vez conforme necessário.<br>
Vamos ver a seguir como fazer esta definição utilizando orientação a objetos.

In [295]:
class SquareIterator():
    
    def __init__(self, max_value):
        
        self.max_value = max_value
        self.current_value = 0
        
    def __iter__(self):
        
        return self
    
    def __next__(self):
        
        if self.current_value >= self.max_value:
            
            raise StopIteration
            
        current_squared_value = self.current_value ** 2
        
        self.current_value += 1
        
        return current_squared_value

In [30]:
for num in SquareIterator(3):
    
    print(num)

0
1
4


In [31]:
for num in calculate_squares(3):
    
    print(num)

0
1
4


Utilizando tanto a classe como a função, quanto a classe eu consigo resolver o problema de iterar sobre uma lista de quadrados de números entre 0 e um valor máximo.
A diferença é que no caso da classe, eu não precisar gerar uma lista completa a cada chamada next, o objeto altera seu estado através do retorno de uma nova instância com seus atributos atualizados até atingir o limite.

Interessante, entramos agora em um novo questionamento...<br>
Ok, interessante essa criação da classe onde definindo os métodos `__iter__` e `__next__` consigo criar meu iterator sem precisar criar de fato uma sequencia inteira, acessando os elementos um a um conforme for preciso.<br>
A dúvida é, existe forma mais fácil e prática de fazer isso?<br>
E a resposta é sim!

### Generator Functions

As generate functions pelo que estou lendo tem muitas funcionalidades, mas a principal explorada neste texto é exatamente a praticidade na geração de iterators.<br>
Estas funções se parecem muito com uma função tradicional mas ao invés de ter um `return` ocorre uma nova palavra-chave, `yield`. E é exatamente este cara que faz toda a diferença.

In [1]:
def gen_nums():
    
    n = 0
    
    while n < 4:
        
        yield n
        
        n += 1

In [4]:
for num in gen_nums():
    
    print(num)

0
1
2
3


In [5]:
sequence = gen_nums()
type(sequence)

generator

Vamos as definições:

- `GENERATOR FUNCTION:` - É uma função na qual eu defino retorno através da palavra-chave yield e não da palavra `return`.

- `GENERATOR OBJECT:` - É o objeto retornado pela GENERATOR FUNCTION, e na real, ele é um iterator onde posso aplicar a built-in function next() e utilizar em for loops por exemplo.

In [10]:
next(sequence)

StopIteration: 

Aqui temos uma definição interessante sobre as GENERATOR FUNCTIONS e sua comparação com as regular functions.<br>
Enquanto funções regulares podem apresentar vários pontos de saída (return statement), elas apresentam apenas um ponto de entrada. Ou seja, toda vez que uma função regular é chamada, ela roda desde sua primeira linha.<br>
No caso das GENERATOR FUNCTIONS, cada yield statement funciona como ponto de saída e entrada. Ou seja, ao chamar a função uma primeira vez, ela roda até o primeiro yield, ao rodar a função novamente ela começa a rodar exatamente na linha seguinte ao yield retornado anteriormente. 

__Obs:__ Não é necessário dar um raise em uma excessão (StopIteration), no caso do GENERATOR OBJECT retornado por uma GENERATOR FUNCTION, ele jé é implementado nativamente.<br>

Vamos fazer alguns testes:

In [11]:
def gen_function():
    
    yield "Rodrigo Bernardo Medeiros"

In [33]:
sequence = gen_function()

In [35]:
next(sequence)

StopIteration: 

Seria uma prática interessande utilizar generator functions para definir fluxos de desenvolvimento, ou para operar parcialmente sobre valores?

In [36]:
def split_names(complete_name):
    
    names = complete_name.split()
    
    for name in names:
        
        yield name

In [42]:
names = [
    'rodrigo bernardo medeiros', 
    'adriane bernardo medeiros', 
    'roberval mauricio cesar'
]

first_name = list()
second_name = list()
third_name = list()

for name in names:
    
    separated_names = split_names(name)
    
    first_name.append(next(separated_names))
    second_name.append(next(separated_names))
    third_name.append(next(separated_names))

In [44]:
first_name

['rodrigo', 'adriane', 'roberval']

In [45]:
second_name

['bernardo', 'bernardo', 'mauricio']

In [46]:
third_name

['medeiros', 'medeiros', 'cesar']

Neste exemplo acima, eu utilizo uma GENERATOR FUNCTION para tratar determinado informação.

Vamos retornar ao exemplo do quadrado dos números, reformulando a solução utilizando uma GENERATOR FUNCTION.

In [47]:
def gen_square_numbers(max_value):
    
    current_value = 0
    
    while current_value < max_value:
        
        yield current_value ** 2
        
        current_value += 1

In [48]:
for num in gen_square_numbers(4):
    
    print(num)

0
1
4
9


Muito bom, neste caso foi possível reescrever a classe escrita anteriormente que definia diretamente um itarator por uma GENERATOR FUNCTION que retorna um GENERATOR OBJECT que no fundo também é um ITERATOR.
Repara-se pela verbosidade de cada código que utilizar GENERATOR FUNCTIONS parece ser muito mais natural, fácil de ler, manter e escalar.

### Generator Patterns and scalable Composability

Agora vamos explorar um pouco mais as GENERATOR FUNCTIONS.

In [123]:
def matching_lines_from_file(path, pattern):
    
    with open(path) as handle:
        
        for line in handle:
            
            if pattern in line:
                
                yield line.rstrip('\n')

Neste exemplo, escrevi uma função para ler um arquivo e retornar um GENERATOR OBJECT que itera sobre as linhas que contém determinado padrão de caracteres.

In [136]:
path = r'./src/log_file.txt'
pattern = 'WARNING'

for line in matching_lines_from_file(path, pattern):
    
    print(line)

Através da GENERATOR FUNCTION, consegui iterar somente pelas linhas de interesse sem precisar trazer para meu código a complexidade de encontrar o padrão. Resolvo isso através do iterator. Posso escrever essas funções como se fossem uma espécie de filtro sem precisar construir uma sequencia para alocar a informação.

### Algumas boas práticas neste desenvolvimento:

- Utilizar `with` para abrir o arquivo, é uma boa prática que garante que o arquivo será fechado assim que o código passar pelo bloco de contexto.

- Neste exemplo, utilizei o modo pythonico de iterar por linhas em um arquivo. Utilizando o for line in file, aproveito este ponto para lembrar de registrar e não mais utilizar o readlines(), este método carrega o arquivo inteiro e com essa joga fora toda minha estratégia de utilizar menos memória em meus desenvolvimentos. Ou seja, jogo fora minha escalabilidade.

Vamos avançar em nossa função para analisar nosso arquivo de log, agora além de pegar somente as linhas com o padrão WARNING, quero também parsear a informação da linha em um dicionário com duas chaves, `level` para escrever o tipo de mensagem de log e `message` para escrever a mensagem efetivamente:


In [137]:
def parse_log_records(lines):
    
    for line in lines:
        
        level, message = line.split(':', 1)
        
        yield {
            "level": level,
            "message": message
        }

In [138]:
valid_lines = matching_lines_from_file(path, pattern)

for parsed_info in parse_log_records(valid_lines):
    
    print(parsed_info)

{'level': 'WARNING', 'message': ' Disk usage exceeding 85%'}
{'level': 'WARNING', 'message': ' Almost out of beer'}


Neste caso combinei duas funções para tratar problemas distintos, ambas são GENERATOR FUNCTIONS que retornam GENERATOR OBJECTS. pelo livro que estou estudando esse tratamento define a composição de escalabilidade. Ou seja, é sempre importante pensar na implementação a nível desses pequenos detalhes, como escalar, como evitar grandes consumos de memória.
Mais um detalhe de implementação, a função matching_lines_from_file, recebe um caminho e padrão para executar duas tarefas, uma ler os arquivos e outra retornar um GENERATOR OBJECT contendo somente as linhas de interesse. Neste ponto vamos fazer uma refatoração para que as interfaces estejam alinhadas e ambas as funções possam receber iteradores e operar sobre eles.

In [143]:
# função responsável por ler um arquivo retornando um GENERATOR OBJECT que itera sobre todas as linhas.

def lines_from_file(path):
    
    with open(path) as handle:
        
        for line in handle:
            
            yield line.rstrip('\n')
            
def matching_lines(lines, pattern):
    
    for line in lines:
        
        if pattern in line:
            
            yield line

In [140]:
lines_from_file = lines_from_file(path)
correct_lines = matching_lines(lines_from_file, 'WARNING')

for line in parse_log_records(correct_lines):
    
    print(line)

{'level': 'WARNING', 'message': ' Disk usage exceeding 85%'}
{'level': 'WARNING', 'message': ' Almost out of beer'}


### Teste utilizando `yield from`

Esta composição de palavras-chave é utilizada quando quero encadear duas generate functions. Vou desenvolver um exemplo baseado na leitura do arquivo com log.

In [150]:
def matching_lines_from_files(path, pattern='WARNING'):
    
    all_lines = lines_from_file(path)
    
    yield from matching_lines(all_lines, pattern)

In [152]:
for line in parse_log_records(matching_lines_from_files(path, pattern='WARNING')):
    
    print(line)

{'level': 'WARNING', 'message': ' Disk usage exceeding 85%'}
{'level': 'WARNING', 'message': ' Almost out of beer'}


### Python está cheio de iteradores

Vamos ver alguns exemplos:

In [167]:
calories = {
    "apple": 95,
    "slice of bacon": 43,
    "cheddar cheese": 113,
    "ice cream": 15
}

items = calories.items()

In [170]:
len(items)

4

In [171]:
iter(items)

In [173]:
('apple', 95) in items

True

In [177]:
for key, value in items: print(f'{key}: {value}')
for ind, item in enumerate(items): print(f'{ind + 1} - {item}')

apple: 95
slice of bacon: 43
cheddar cheese: 113
ice cream: 15
1 - ('apple', 95)
2 - ('slice of bacon', 43)
3 - ('cheddar cheese', 113)
4 - ('ice cream', 15)


Além desse exemplo de iterador, no livro são citados diversos outros exemplos de iterators em python, como os resultados da utilização de map, filter e zip. Vou codar um pouco com esses caras pra ver no que dá.

In [234]:
str_numbers = ['1', '2', '3', '4']

def change_type(number):
    
    return int(number)

int_numbers = map(change_type, str_numbers)

type(int_numbers)

map

A aplicação de uma função map, retorna um objeto do tipo map que adivinhem só, é um iterator.

In [235]:
def filter_test(number):
    
    if number > 2:
        
        return number
    

filter_numbers = filter(filter_test, int_numbers)
type(filter_numbers)

filter

Fazendo um paralelo com a aplicação do map, ao aplicar o filter obtenho um objeto do tipo filter que também é um iterator. É importante saber que isso existe e sempre pensar nesse gerenciamento de memória ao desenvolver as soluções com isso vamos ficando cada vez mais afiados. Lembrando que enquanto a gente só conhece martelo, todo problema é prego. Conhecer novas formas de resolver problemas nos torna mais criativos

### Iterator protocol

objetos que são considerados iterators se seguem o seguinte protocolo:

1) Tem a implementação de um método `__next__` que não tem parametros.<br>
2) O método `__next__` é responsável por atualizar o estado do objeto para o próximo item na sequencia.<br>
3) Após todos os elementos serem produzidos, o método `__next__` é responsável por dar um raise exception no erro StopIteration.<br>
4) Ocorre a definição de um método `__iter__` que não recebe parametros e retorna o próprio elemento. Liretamente um método com um return self dentro.<br>

### Definição de um objeto iterável.

In [240]:
import pandas as pd

In [264]:
content = {
    'names': [
        'rodrigo', 
        'renata',
        'isabella', 
        'gabriella'
    ],
    'age': [
        35,
        34,
        2,
        0
    ]
}

family = pd.DataFrame(content)

In [265]:
family.to_dict(orient='records')

[{'names': 'rodrigo', 'age': 35},
 {'names': 'renata', 'age': 34},
 {'names': 'isabella', 'age': 2},
 {'names': 'gabriella', 'age': 0}]

In [285]:
for ind, item in family.iterrows():
    
    print(dict(item))

{'names': 'rodrigo', 'age': 35}
{'names': 'renata', 'age': 34}
{'names': 'isabella', 'age': 2}
{'names': 'gabriella', 'age': 0}


In [287]:
family.loc[0]

names    rodrigo
age           35
Name: 0, dtype: object

In [288]:
class DataFrame():
    
    def __init__(self, dataframe):
        
        self._dataframe = dataframe.reset_index(drop=True)
        self._index = 0
        
    def __iter__(self):
        
        return self
    
    def __next__(self):
        
        try:
            
            return dict(self._dataframe.loc[self._index])
        
        except:
            
            raise StopIteration
        
        finally:
            
            self._index += 1

In [289]:
# implementing __iter__ metho to define an iterable object, this method must return a iterator in this case I just apply 
# the built-in function iter() in the list.

class IterRowsDataFrame():
    
    def __init__(self, dataframe):
        
        self._dataframe = dataframe
        
    def __iter__(self):
        
        return iter(self._dataframe.to_dict(orient='records'))

In [290]:
# Just to solve the problem in othet way, in this case instead of use iter(), I defined the __iter__ method as a generator
# function which returns a generator object.

class IterRowsDataFrameGeneratorFunction():
    
    def __init__(self, dataframe):
        
        self._dataframe = dataframe
        
    def __iter__(self):
        
        index = 0
        
        while index < len(self._dataframe):
        
            yield self._dataframe.to_dict(orient='records')[index]
            
            index += 1

In [291]:
dict_rows_family = DataFrame(family)
for item in dict_rows_family: print(item)

{'names': 'rodrigo', 'age': 35}
{'names': 'renata', 'age': 34}
{'names': 'isabella', 'age': 2}
{'names': 'gabriella', 'age': 0}


In [292]:
dict_rows_family = IterRowsDataFrame(family)
for item in dict_rows_family: print(item)

{'names': 'rodrigo', 'age': 35}
{'names': 'renata', 'age': 34}
{'names': 'isabella', 'age': 2}
{'names': 'gabriella', 'age': 0}


In [293]:
dict_rows_family = IterRowsDataFrameGeneratorFunction(family)
for item in dict_rows_family: print(item)

{'names': 'rodrigo', 'age': 35}
{'names': 'renata', 'age': 34}
{'names': 'isabella', 'age': 2}
{'names': 'gabriella', 'age': 0}


In [277]:
teste = DataFrame(family)
next(teste)

{'names': 'rodrigo', 'age': 35}

In [278]:
teste = IterRowsDataFrame(family)
next(teste)

TypeError: 'IterRowsDataFrame' object is not an iterator

In [279]:
teste = IterRowsDataFrameGeneratorFunction(family)
next(teste)

TypeError: 'IterRowsDataFrameGeneratorFunction' object is not an iterator

Como podemos ver nestes exemplos, a classe DataFrame é um iterator pela definição dos métodos necessários (`__iter__`, `__next__`), enquanto que as demais são apenas iterables definindo somente o método `__iter__`.

# CREATING COLLECTIONS WITH COMPREHENSIONS

# CRIANDO COLEÇÕES COM COMPREENSÕES

List comprehension é uma forma declarativa de alto nível para criar listas em python.

In [1]:
squares = [n * n for n in range(6)]
print(squares)

[0, 1, 4, 9, 16, 25]


Isto é exatamente equivalente ao seguinte:

In [2]:
squares = list() # ou squares = []

for n in range(6):
    
    squares.append(n * n)
    
print(squares)

[0, 1, 4, 9, 16, 25]


Observe que no primeiro exemplo, o que você digita está declarando que tipo de lista você quer, enquanto que no segundo está especificando como cria-la. 
Por isso que nós dizemos que é uma forma alto nível e declarativa:Funciona como se vocês estivesse definindo que tipo de lista quer criar, e depois, deixa o Python descobrir como construi-la. 

Python permite escrever outros tipos de `comprehensions` além das listas. Segue um exemplo de `dictionary comprehension`:

In [3]:
blocks = { n: "x" * n for n in range(5)}
print(blocks)

{0: '', 1: 'x', 2: 'xx', 3: 'xxx', 4: 'xxxx'}


Isto é exatamente equivalente a fazer o seguinte:

In [4]:
blocks = dict()

for n in range(5):
    
    blocks[n] = 'x' * n
    
print(blocks)

{0: '', 1: 'x', 2: 'xx', 3: 'xxx', 4: 'xxxx'}


Os principais benefícios das `comprehensions` são legibilidade e manutenibilidade. Muitas pessoas, acham elas muito legíveis; Mesmo desenvolvedores que nunca encontraram elas antes, podem na maioria das vezes adivinhar corretamente o que significa.
E além disso, há um benefício cognitivo mais profundo: Uma vez que você praticar um pouco, vai achar que pode escreve-las com pouquíssimo esforço mental - mantendo sua atenção livre para outras tarefas.

## List Comprehensions

__`List comprehension` é a mais usada e util tipo de `comprehension`, e essencialmente é uma forma de criar e popular uma lista.__ Sua estrutura segue a seguinte expressão:

[ __EXPRESSION__ for __VARIABLE__ in __SEQUENCE__ ]

*EXPRESSION* é qualquer expressão python, embora em compreensões, esta expressão tipicamente contenha uma variável. Esta variável é declarada no campo *VARIABLE*. *SEQUENCE* define a fonte dos valores pelos quais a variável itera, criando uma sequencia final de valores calculados.

In [17]:
squares = [n * n for n in range(6)]
print(type(squares))
print(squares)

<class 'list'>
[0, 1, 4, 9, 16, 25]


Observe que o resultado é uma lista regular. Em __squares__, a expressão é n\*n; a variável é n; e a sequencia fonte é range(6). A sequencia é um objeto do tipo range; Na verdade, neste ponto podemos utilizar qualquer iterável, outra lista ou tuple, um generator object, ou qualquer outra coisa.
<br>
A parte da expressão pode ser qualquer que se reduz a um valor:
<br>
- Expressões aritiméticas
- Uma chamada de função do tipo f(m), utilizando m como variável.
- Uma operação de corte (como s[::-1], para reverter uma string).
- Chamadas de métodos (foo.bar(), iterando sobre uma sequencia de objetos).
- E muito mais...

__Alguns exemplos:__

In [21]:
# Primeiro definimos algumas sequencias fonte:

pets = 'dog parakeet cat llama'.split()
numbers = [9, -1, -4, 20, 11, -3]

# Uma função para chamar
def repeat(s):
    
    return s + '_' + s

# Agora, alguns exemplos de list comprehensions:

print([2 * m + 3 for m in range(10, 20, 2)])
print(sorted([abs(num) for num in numbers]))
print([10 - x for x in numbers])
print([pet.lower() for pet in pets])
print(["The " + pet for pet in pets])
print([repeat(pet) for pet in pets])

[23, 27, 31, 35, 39]
[1, 3, 4, 9, 11, 20]
[1, 11, 14, -10, -1, 13]
['dog', 'parakeet', 'cat', 'llama']
['The dog', 'The parakeet', 'The cat', 'The llama']
['dog_dog', 'parakeet_parakeet', 'cat_cat', 'llama_llama']


Observe como todas se encaixam na mesma estrtura. Todos tem as palavras-chave __for__ e __in__; Que são requeridas  pelo Python, em qualquer tipo de compreensão que você possa escrever. Elas são intercaladas entre três campos: a expressão; a variável; e a sequencia fonte.<br>

A ordem dos elementos na lista final é determinado pela ordem da sequencia fonte. Porém é possível filtrar adicionando uma clausula *if*.

In [24]:
def is_palindrome(s):
    
    return s == s[::-1]

pets = 'dog parakeet cat llama'.split()
numbers = [9, -1, -4, 20, 11, -3]
words = 'bib bias dad eye deede tooth'.split()

print([n*2 for n in numbers if n % 2 == 0])
print([pet.upper() for pet in pets if len(pet) == 3])
print([n for n in numbers if n >0])
print([word for word in words if is_palindrome(word)])

[-8, 40]
['DOG', 'CAT']
[9, 20, 11]
['bib', 'dad', 'eye']


A estrutura é:

[EXPR for VAR in SEQUENCE if CONDITION]

Onde *CONDITION* é a expressão que avalia para Verdadeiro ou Falso, dependendo da variável. Note que também podemos utilizar uma função aplicada a variável (is_palindrome(word)), ou algo mais complexo (len(pet) == 3). Optando por usar uma função pode melhorar a legibilidade, e também permite aplicar uma lógica de filtro que não se encaixa em somente uma linha.

Uma `list comprehension` sempre deve ter a palavra "for", mesmo que a expressão seja a própria variável. Por exemplo, quando dizemos:

In [25]:
[word for word in words if is_palindrome(word)]

['bib', 'dad', 'eye']

As vezes as pessoas pensam que `word for word in words` parece redundante (e é), e tentam encurtar a sentença. Porém, isso não funciona, como apresentado no exemplo a seguir:

In [26]:
[word in word if is_palindrome(word)]

SyntaxError: invalid syntax (<ipython-input-26-42bedfba4293>, line 1)

### Formatando Para Legibilidade (e mais)

`List Comprehensions` realisticas, tendem ser muito longas para encaixar bem em apenas uma linha. E elas são compostas de partes lógicas distintas, que podem variar independentemente conforme o código evolui. Isto cria alguns inconvenientes, que são resolvidos por um fato muito conveniente: As regras normais de espaços em branco são suspensas dentro de colchetes. Você pode explorar isso e fazer com as `list comprehensions` sejam mais legíveis e manuteníveis, divindo-as através de múltiplas linhas.

In [27]:
def double_short_words(words):
    
    return [
        word + '_' + word
        for word in words
        if len(word) < 5
    ]

In [28]:
print(double_short_words(words))

['bib_bib', 'bias_bias', 'dad_dad', 'eye_eye']


O que foi feito aqui foi dividir a `comprehension` através de linhas separadas. Você pode, e deve, fazer isso com qualquer `comprehension`. Isso é ótimo por várias razões, a mais importante sendo o ganho instantâneo na legibilidade. Esta `comprehension` tem três ideias separadas expressas dentro dos colchetes: a expressão (word + word); a sequencia (for word in words); e a clausula de filtro (if len(word) < 5). Estes são aspectos lógicos separados, e fazendo a divisão através das linhas, toma-se menos esforço cognitivo para um humano ler e entender quando comparado a versão escrita em somente uma linha. Com isso a lista está previamente parseada para você, conforme você lê o código.

Existe um outro benefício: Controle de versão e revisão de código são mais fáceis de se fazer. Imagine que você e eu somos desenvolvedores no mesmo time, trabalhando na mesma base de código em diferentes branchs. No meu branch, eu altero a expressão `word + '_' + word` para `word + '_' + word + '_' + word`; no seu, você muda o limiar para "len(word) < 7". Se a `comprehension` estiver na mesma linha, as ferramentas de versionamento de código vão identificar isso como um conflito de fusão, e estes conflitos terão que ser resolvidos manualmente. Mas uma vez que a `list comprehension` esteja dividida entre várias linhas, nossa ferramenta de controle de versão vai automaticamente ser capaz de juntar todas as modificações sem conflito. E se nós estivermos fazendo uma revisão de código, como deveriamos, o revisor pode identificar com precisão a mudança imediatamente, sem ter que percorrer linhas por linha pensando.

### Múltiplas Fontes e Filtros

Você pode ter várias clausulas `for VAR in SEQUENCE`. Isto permite que você construa listas com pares, trios, etc... a partir de uma ou mais sequências fonte.

In [31]:
colors = 'orange purple pink'.split()
toys = 'bike basketball skateboard doll'.split()

[
    color + '_' + toy
    for color in colors
    for toy in toys
]

['orange_bike',
 'orange_basketball',
 'orange_skateboard',
 'orange_doll',
 'purple_bike',
 'purple_basketball',
 'purple_skateboard',
 'purple_doll',
 'pink_bike',
 'pink_basketball',
 'pink_skateboard',
 'pink_doll']

Todo par vindo dessas duas fontes, *colors* e *toys*, é utilizado para calcular um valor na lista final. Esta lista final tem 12 elementos, o produto da dimensão das duas listas fonte.

Eu quero que você note que as duas clausulas `for` são independentes entre si; *colors* e *toys* são duas listas não relacionadas. Utilizar múltiplas clausulas `for` pode as vezes tomar uma forma diferente, onde elas são mais independentes.

Considere este exemplo:

In [40]:
ranges = [range(1, 7), range(7, 12), range(12, 21)]

[
    float(num)
    for subrange in ranges
    for num in subrange
]

[1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 20.0]

A sequencia fonte - "ranges" - é uma lista de objetos do tipo range. Agora, esta `list comprehension` tem duas clausulas novamente. Mas observe que uma depende da outra. A fonte da segunda é uma variável da primeira!

Este exemplo não é como o exemplo de brinquedos coloridos, onde as clausulas são independentes umas das outras. Quando encadeados desta forma a ordem é importante.

In [41]:
[
    float(num)
    for num in subrange
    for subrange in ranges
]

NameError: name 'subrange' is not defined

Python avalia a `list comprehension` da esquerda para a direita. Se a primeira clausula é `for num in subrange`, neste momento, subrange não está definido ainda. Então, você deve colocar `for surange in ranges` primeiro. Você pode encadear mais que duas clausula `for` juntas; a primeira vai somente referenciar uma fonte definida anteriormente, e as outras podem usar as fontes definidas nas clausulas `for` anteriores, como a subrange mostrada no exemplo.

Beleza, isto é para clausulas `for` encadeadas. Se as clausulas são independentes, a ordem importa? A resposta é sim, importam porém de uma forma diferente. Qual a diferença entre essas duas duas `list comprehensions`.

In [42]:
colors = 'orange purple pink'.split()
toys = 'bike basketball skateboard doll'.split()

In [49]:
[
    color + ' ' + toy
    for color in colors
    for toy in toys
]

['orange bike',
 'orange basketball',
 'orange skateboard',
 'orange doll',
 'purple bike',
 'purple basketball',
 'purple skateboard',
 'purple doll',
 'pink bike',
 'pink basketball',
 'pink skateboard',
 'pink doll']

In [50]:
[
    color + ' ' + toy
    for toy in toys
    for color in colors
]

['orange bike',
 'purple bike',
 'pink bike',
 'orange basketball',
 'purple basketball',
 'pink basketball',
 'orange skateboard',
 'purple skateboard',
 'pink skateboard',
 'orange doll',
 'purple doll',
 'pink doll']

A ordem aqui não importa na mesma forma que as clausulas `for` encadeadas, onde você tem que colocar as coisas em certa ondem, ou seu programa não vai rodar. Aqui, você tem uma escolha. E esta escolha a ordem dos elementos na lista final. O primeiro elemento em cada um dos arranjos é "orange bike". E observe que o segundo elemento é diferente. Pense um momento, e pergunte a você mesmo: Por que? Por que o primeiro elemento é o mesmo em ambas as listas? E porque somente a partir do segundo elemento o arranjo é diferente?

Isto tem haver com qual sequencia é mantida constante enquanto a outra varia. É a mesma lógica que aplicamos em loops `for` aninhados.

In [52]:
# Primeira configuração:

build_colors_toys = []

for color in colors:
    
    for toy in toys:
        
        build_colors_toys.append(color + " " + toy)
        
        
print(build_colors_toys[0])
print(build_colors_toys[1])

orange bike
orange basketball


In [53]:
# Segunda configuração:

build_toys_colors = []

for toy in toys:
    
    for color in colors:
        
        build_toys_colors.append(color + " " + toy)
        
print(build_toys_colors[0])
print(build_toys_colors[1])

orange bike
purple bike


A segunda clausula `for` na `list comprehension` corresponde ao `for loop` mais interno. Os valores variam através de seu range mais rapidamente do que o loop externo.

E além de poder utilizar múltiplas clausulas `for`, você pode ter mais de uma clausula `if`, para multiplos níveis de filtro. Basta escrever várias clausulas em sequencia.

In [73]:
numbers = [9, -1, -4, 20, 17, -3]

odd_positives = [
    num for num in numbers
    if num % 2 == 1
    if (num >0)
]

print(odd_positives)

[9, 17]


Aqui, eu coloquei cada clausula `if` em sua própria linha, pela legibilidade - mas eu poderia colocar ambas em uma linha apenas. Quando você tiver mais de uma clausula `if`,  cada elemento deve atender todos os critérios para fazer parte da lista final. Em outras palavras, clausulas `if` são do tipo `and` e não `or`.

E se você quiser fazer uma clausula `or` - para incluir elementos que atendem pelo menos uma das clausulas `if`, omitindo apenas aquelas que não atendem ambas? `list comprehension` não permite fazer isso diretamente. A mini-linguagem para escrever `comprehensions` não é tão expressiva quando o próprio Python, e existem listas que você precisa construir que não podem ser expressas como uma `comprehension`.

Mas as vezes, você pode contornar um pouco através da definição de funções. Por exemplo, como você poderia construir um filtro baseado na clausula de um número ser múltiplo de 2 __ou__ 3.

In [74]:
numbers = [9, -1, -4, 20, 11, -3]

def is_mul_of_2_or_3(num):
    
    return (num % 2 == 0) or (num % 3 == 0)

[
    num for num in numbers
    if is_mul_of_2_or_3(num)
]

[9, -4, 20, -3]

Uma outra alternativa é escrever uma clausula `if` mais complexa, composta por múltiplos testes.

In [77]:
[
    num for num in numbers
    if 
    (num % 2 == 0) |
    (num % 3 == 0)
]

[9, -4, 20, -3]

Essa limitações serão discutidas melhor posteriormente.

Podemos utilizar multiplas clausulas `for` e `if` juntas:

In [80]:
weights = [0.2, 0.5, 0.9]
values = [27.5, 13.4]
offsets = [4.3, 7.1, 9.5]

from collections import namedtuple

infos = namedtuple('WeightInfo', 'weigth value offset')

[
    infos(weight, value, offset)
    for weight in weights
    for value in values
    for offset in offsets
    if offset >5.0
    if weight * value < offset
]

[WeightInfo(weigth=0.2, value=27.5, offset=7.1),
 WeightInfo(weigth=0.2, value=27.5, offset=9.5),
 WeightInfo(weigth=0.2, value=13.4, offset=7.1),
 WeightInfo(weigth=0.2, value=13.4, offset=9.5),
 WeightInfo(weigth=0.5, value=13.4, offset=7.1),
 WeightInfo(weigth=0.5, value=13.4, offset=9.5)]

In [12]:
def double_short_words(words):
    
    return [
        word + word
        for word in words
        if len(word) < 5
    ]

In [16]:
words = 'ola ole ovo rodrigo bernardo medeiros'.split()

double_small_words = double_short_words(words)
print(double_small_words)

['olaola', 'oleole', 'ovoovo']
